In [1]:
# importing required libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
# importing datasets
train_dir = "../input/emotion-detection-fer/train" # Replace with appropriate path
test_dir = "../input/emotion-detection-fer/test"  

In [3]:
img_size=48

In [4]:
# data augmentation
train_datagen = ImageDataGenerator(
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    rescale = 1./255,
    validation_split = 0.2,
    rotation_range = 30
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    directory = train_dir,
    target_size = (img_size,img_size),
    shuffle = True,
    batch_size = 64,
    color_mode = "grayscale",
    class_mode = "categorical",
    subset = "training"
)

validation_generator = train_datagen.flow_from_directory(
    directory = train_dir,
    target_size = (img_size,img_size),
    shuffle = True,
    batch_size = 64,
    color_mode = "grayscale",
    class_mode = "categorical",
    subset = "validation"
)

test_generator = validation_datagen.flow_from_directory(
    directory = test_dir,
    shuffle = True,
    target_size = (48, 48),
    batch_size = 64,
    color_mode = 'grayscale',
    class_mode = 'categorical'
)

In [9]:
# CNN model 1
model1 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=5, activation='relu', padding='SAME', input_shape=(48, 48, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=64, kernel_size=5, activation='relu', padding='SAME'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='SAME'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='SAME'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(filters=256, kernel_size=5, activation='relu', padding='SAME'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(filters=256, kernel_size=5, activation='relu', padding='SAME'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='SAME'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer='l2'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1024, activation='relu', kernel_regularizer='l2'),
    tf.keras.layers.Dropout(0.2),

    # output layer
    tf.keras.layers.Dense(units=7, activation='softmax')
])

model1.summary()

model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
epochs = 45
batch_size = 64

In [11]:
# training model 1
model1.fit(train_generator,epochs = epochs, validation_data = validation_generator)

In [12]:
# evaluating model 1
model1.evaluate(test_generator)

In [13]:
# saving the model in .json file
model_json = model1.to_json()
with open('my_model1.json', 'w') as file:
    file.write(model_json)

# saving weights in .h5 file
model1.save_weights('my_model1.h5')

In [14]:
# CNN model 2
my_AlexNet = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=5, activation='relu', padding='SAME', input_shape=(48, 48, 1)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='SAME'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='SAME'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='SAME'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='SAME'),
    tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='SAME'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='SAME'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(units=512, activation='relu', kernel_regularizer='l2'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1024, activation='relu', kernel_regularizer='l2'),
    tf.keras.layers.Dropout(0.2),

    # output layer
    tf.keras.layers.Dense(units=7, activation='softmax')
])

my_AlexNet.summary()

my_AlexNet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
# training model 2
my_AlexNet.fit(train_generator,epochs = epochs, validation_data = validation_generator)

In [16]:
# evaluating model 2
my_AlexNet.evaluate(test_generator)

In [17]:
# saving the model in .json file
model_json = my_AlexNet.to_json()
with open('my_AlexNet.json', 'w') as file:
    file.write(model_json)

# saving weights in .h5 file
my_AlexNet.save_weights('my_AlexNet.h5')